In [1]:
import keras
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
# example of loading the resnet50 model
from keras.applications.resnet50 import ResNet50
# load model
resnet = ResNet50(weights = 'imagenet', input_shape = (224,224,3))
# summarize the model
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [3]:
for layer in resnet.layers:
    layer.trainable = False
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [4]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers

model = Sequential()

model.add(resnet)
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=2e-5),
metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1000)              25636712  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               512512    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 2565      
Total params: 26,414,445
Trainable params: 777,733
Non-trainable params: 25,636,712
____________________________________

In [5]:
trdata = ImageDataGenerator(rescale=1./255, rotation_range = 90, zoom_range = 0.3)
traindata = trdata.flow_from_directory(directory=r"C:/Users/Meeta Malviya/Videos/data2/train",target_size=(224,224))
tsdata = ImageDataGenerator(rescale=1./255, rotation_range = 90, zoom_range = 0.3)
testdata = tsdata.flow_from_directory(directory=r"C:/Users/Meeta Malviya/Videos/data2/test", target_size=(224,224))

Found 1023 images belonging to 5 classes.
Found 200 images belonging to 5 classes.


In [6]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("resnet50.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')

history = model.fit_generator(generator=traindata, steps_per_epoch=2, epochs=100,
                                validation_data=testdata,
                                validation_steps=50,
                                verbose=1,callbacks=[checkpoint,early])
model_final.save_weights("resnet50.h5")



Epoch 1/100
2/2 [==============================] - 855s 427s/step - loss: 0.6932 - accuracy: 0.5281 - val_loss: 0.6930 - val_accuracy: 0.6018
Epoch 2/100


C:\Users\Meeta Malviya\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
C:\Users\Meeta Malviya\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


2/2 [==============================] - 532s 266s/step - loss: 0.6928 - accuracy: 0.5873 - val_loss: 0.6925 - val_accuracy: 0.7996
Epoch 3/100
2/2 [==============================] - 518s 259s/step - loss: 0.6923 - accuracy: 0.7063 - val_loss: 0.6923 - val_accuracy: 0.8000
Epoch 4/100
2/2 [==============================] - 516s 258s/step - loss: 0.6923 - accuracy: 0.6844 - val_loss: 0.6922 - val_accuracy: 0.8000
Epoch 5/100
2/2 [==============================] - 717s 358s/step - loss: 0.6923 - accuracy: 0.7094 - val_loss: 0.6919 - val_accuracy: 0.8000
Epoch 6/100
2/2 [==============================] - 593s 296s/step - loss: 0.6919 - accuracy: 0.7094 - val_loss: 0.6916 - val_accuracy: 0.8000
Epoch 7/100
2/2 [==============================] - 766s 383s/step - loss: 0.6915 - accuracy: 0.7688 - val_loss: 0.6917 - val_accuracy: 0.8000
Epoch 8/100
2/2 [==============================] - 817s 408s/step - loss: 0.6916 - accuracy: 0.7437 - val_loss: 0.6913 - val_accuracy: 0.8000
Epoch 9/100
2/2 [=

KeyboardInterrupt: 

In [7]:
from sklearn.externals import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(model, 'restnet50_model_data2.pkl') 

C:\Users\Meeta Malviya\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['restnet50_model_data2.pkl']